In [3]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import os
import numpy as np

# File paths
street_data_path = "/content/drive/MyDrive/CS524/crime_data_2013_onward.csv"  # Crime data
street_shapefile_path = "/content/drive/MyDrive/CS524/LVI/transportation_20241202.csv"  # Street shapefile data
lpi_files_path = "/content/drive/MyDrive/CS524/LPI/"  # LPI files
geojson_output_folder = "/content/drive/MyDrive/CS524/LVI/GeoJSON/"  # GeoJSON files
crime_lvi_files_path = "/content/drive/MyDrive/CS524/LVI/crime/"  # LVI files
streetlight_outages_path = "/content/drive/MyDrive/CS524/streetlight_outages.csv"

# street data with id and block
street_data = pd.read_csv(street_data_path)[['id', 'block']]
shapefile_data = pd.read_csv(street_shapefile_path)


# filter streetlight outage data with creation and completion dates
outage_data = pd.read_csv(streetlight_outages_path)
outage_data['Creation Date'] = pd.to_datetime(outage_data['Creation Date'])
outage_data['Completion Date'] = pd.to_datetime(outage_data['Completion Date'])

In [4]:
os.makedirs(geojson_output_folder, exist_ok=True)

# Combine columns to get street name
shapefile_data['street_name'] = (
    shapefile_data['PRE_DIR'].str.strip() + " " +
    shapefile_data['STREET_NAM'].str.strip() + " " +
    shapefile_data['STREET_TYP'].str.strip()
).str.upper()

shapefile_data['geometry'] = shapefile_data['the_geom'].apply(wkt.loads)
street_gdf = gpd.GeoDataFrame(shapefile_data, geometry='geometry', crs='EPSG:4326')

# function to assign streetlight outages to crimes in a 100m threshold
def assign_streetlight_outage(crime_date, crime_location):
    """
    Assigns a streetlight outage (1 for outage, 0 for no outage) based on proximity and active outage dates.
    """
    active_outages = outage_data[
        (outage_data['Creation Date'] <= crime_date) & (outage_data['Completion Date'] >= crime_date)
    ]

    for _, outage in active_outages.iterrows():
        outage_location = (outage['Latitude'], outage['Longitude'])
        distance = geodesic(crime_location, outage_location).meters

        if distance <= 100:
            return 1
    return 0

In [ ]:

import json
# Process each year's LPI Excel file
for year in range(2013, 2024):
    lpi_file_path = os.path.join(lpi_files_path, f"light_pollution_crime_data_{year}.xlsx")
    if not os.path.exists(lpi_file_path):
        print(f"File not found: {lpi_file_path}, skipping...")
        continue

    print(f"Processing {year}...")

    lpi_data = pd.read_excel(lpi_file_path)
    merged_data = pd.merge(lpi_data, street_data, left_on="crime_id", right_on="id", how="left")

    merged_data['street_light_outage'] = merged_data.apply(
        lambda row: assign_streetlight_outage(row['date'], (row['latitude'], row['longitude'])), axis=1
    )

    merged_data['street'] = merged_data['block'].str[5:].str.strip().str.upper()

    output_path = os.path.join(crime_lvi_files_path, f"crime_final_{year}.csv")
    merged_data.to_csv(output_path, index=True)

    # Group data to calculate LVI
    street_lvi = merged_data.groupby('street').apply(
        lambda x: pd.Series({
            'LVI': 0.7 * x['light_pollution_index'].mean() + 0.3 * x['street_light_outage'].mean(),
            'LPI': x['light_pollution_index'].mean(),
            'Street_Light_Outage': x['street_light_outage'].mean(),
            'Top_Crime': x['primary_type'].mode()[0] if not x['primary_type'].mode().empty else None,
            'Crime_Count': len(x),
            'Frequent_Time': x['date'].dt.hour.mode()[0] if not x['date'].dt.hour.mode().empty else None,
            'monthly_crime_counts': json.dumps(x['date'].dt.month.value_counts().to_dict())
        })
    ).reset_index()

    # Merge the street-level LVI with shapefile GeoDataFrame and generate a geoJSON
    street_lvi['street_name'] = street_lvi['street']
    merged_gdf = street_gdf.merge(street_lvi, on='street_name', how='left')

    geojson_output_path = os.path.join(geojson_output_folder, f"street_lvi_{year}.geojson")
    merged_gdf.to_file(geojson_output_path, driver="GeoJSON")
    print(f"GeoJSON file for {year} saved to {geojson_output_path}")

Processing 2013...


<ipython-input-5-f8bb748309cc>:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  street_lvi = merged_data.groupby('street').apply(


GeoJSON file for 2013 saved to /content/drive/MyDrive/CS524/LVI/GeoJSON/street_lvi_2013.geojson
Processing 2014...


<ipython-input-5-f8bb748309cc>:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  street_lvi = merged_data.groupby('street').apply(


GeoJSON file for 2014 saved to /content/drive/MyDrive/CS524/LVI/GeoJSON/street_lvi_2014.geojson
Processing 2015...


...

GeoJSON file for 2021 saved to /content/drive/MyDrive/CS524/LVI/GeoJSON/street_lvi_2021.geojson
Processing 2022...


<ipython-input-5-f8bb748309cc>:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  street_lvi = merged_data.groupby('street').apply(


GeoJSON file for 2022 saved to /content/drive/MyDrive/CS524/LVI/GeoJSON/street_lvi_2022.geojson
Processing 2023...


<ipython-input-5-f8bb748309cc>:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  street_lvi = merged_data.groupby('street').apply(


GeoJSON file for 2023 saved to /content/drive/MyDrive/CS524/LVI/GeoJSON/street_lvi_2023.geojson
